## Sesión 5. Ejemplo de aprendizaje supervisado: regresión.

## Predicción de generación fotovoltaica para autoconsumo.

**Objetivo:** Predecir la generación FV del día siguiente de un hogar, para gestionar de forma inteligente su consumo. Utilizaremos datos históricos de la variable objetivo que queremos predecir (datos históricos de generación fotovoltaica) y otras características que pueden ayudar a predecir el modelo, como la irradiancia o la temperatura.

### Antes de empezar:

* En el archivo **S5_examplePV1.csv** está el conjunto de datos de entrada para este ejemplo (atributos + etiqueta). 

## **1. Importar librerías y datos**.

In [ ]:
# Importamos librerías
import sklearn
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

#importamos dataset
dataset = pd.read_csv('Data/S5_ejemploPV1.csv', delimiter=';')

## **2. Entender los datos**

Es necesario visualizar y entender los datos con los que vamos a trabajar, así como conocer sus características. 

1. ¿Cuántas filas tenemos? Cuántos atributos hay en los datos?  
2. Cuáles son esos atributos?
3. ¿Faltan datos?
4. Resumen estadístico del conjunto de datos de entrada.

**¿Cuántos atributos hay en los datos?**Ç

In [ ]:
### Dataset shape
dataset.shape

**¿Qué significan?**

In [ ]:
# primeras cinco filas
dataset.head()

In [ ]:
# data format
dataset.dtypes

In [ ]:
# Convertir localhour en datetime
dataset['localhour'] = pd.to_datetime(dataset['localhour'])

**3. ¿Falta algún dato? Se comprueba si falta algún dato y, en caso afirmativo, se realiza el recuento de celdas vacías en cada atributo. En este caso, no falta ningún dato.**

In [ ]:
# Check for missing data
dataset.isna().sum()

**4. Resumen estadístico de los datos.**

In [ ]:
dataset.describe()

In [ ]:
from pandas_profiling import ProfileReport
##pandas profiling
#apply ProfileReport
profile = ProfileReport(dataset, title='Profile Report')

In [ ]:
#profile.to_file("your_report2.html")

## Visualizar los datos.

Una forma visual de entender los datos de entrada. 

1. Boxplots y gráficos de densidad
2. Matriz de correlación

### 3.1 Boxplots

El diagrama boxplot nos permite identificar valores atípicos y comparar distribuciones. Además, sabemos cómo se distribuye el 50% de los valores (dentro de la caja).

In [ ]:
%matplotlib inline

In [ ]:
atributos_boxplot = dataset.plot(kind='box', subplots=True, layout=(3, 3), figsize=(15, 10), sharex=False,
                                 sharey=False, fontsize=10)
plt.show()

**Añadir otros gráficos que conozcas**

### **Matriz de correlación** 

**3.2. Cuadro de atributos de doble entrada**

In [ ]:
### Seaborn visualization library
import seaborn as sns

# Calculation of correlation coefficients
corr = dataset.corr(method='pearson') 
# Remove repeated values
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
  
f, ax = plt.subplots(figsize=(12, 10))
#Generar Heat Map,
sns.heatmap(corr, annot=True, fmt=".2f" , mask=mask,)
    # xticks
plt.xticks(range(len(corr.columns)), corr.columns);
    # yticks
plt.yticks(range(len(corr.columns)), corr.columns)
    # plot
plt.show()

## *4. Preparar los datos*.

1. Limpieza de datos
2. Transformación

**1. Limpieza de datos**

No hay Nan en los datos de entrada y no se eliminarán valores atípicos en este ejemplo. 

**2. Transformación**. 

Añado las columnas ``time`` y ``month`` a través de la columna datetime. 
Los datos se escalan

In [ ]:
# Add month and time columns
#dataset['month'] = pd.DatetimeIndex(dataset['localhour']).month
#dataset['hour'] = pd.DatetimeIndex(dataset['localhour']).hour
#dataset.drop(['localhour'], axis=1, inplace=True)
dataset

Divido los datos en **atributos**: X (características) y **etiquetas**: y (objetivo).

In [ ]:
# Features X ; Target y 
# X = dataset.drop(['pvgen'], axis=1) 
# y = dataset['pvgen']

Los datos se escalan utilizando el método ``MinMaxScaler()``, que escala y traduce cada atributo individualmente de forma que esté dentro del rango [0, 1]. Esto debe hacerse cuando las escalas de los atributos son diferentes (por ejemplo, radiación [0, 650], velocidad del viento [2, 15]).

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# I scale attributes/features
scaler = MinMaxScaler()
X_df = X.copy()
X_scaled = pd.DataFrame(scaler.fit_transform(X_df))
X_scaled.columns = X_df.columns
X_scaled.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

## new scaler instance
scaler_y = MinMaxScaler()

# I scale the target/label
y_df = y.copy()
y_df = pd.DataFrame(y_df)
y_scaled = scaler_y.fit_transform(y_df)
# y_scaled.columns = y_df.columns
y_scaled = np.ravel(y_scaled)
y_scaled

## *5. Dividir los datos*.

Los datos se dividen en datos de entrenamiento ``X_train``, ``y_train``, datos de validación ``X_val``, ``y_val`` y datos de prueba ``X_test``, ``y_test``.


In [ ]:
from sklearn.model_selection import train_test_split

train_size =   # percentage of the input data that I will use to validate the model

# I divide the data into training, validation and test data.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, train=train_size,
                                                    shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=train_size,
                                                    shuffle=False)

## *6. Construcción y evaluación de modelos*.

* Las métricas de evaluación seleccionadas son **RMSE y R2**. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

num_folds = 5
error_metrics = {'neg_root_mean_squared_error', 'r2'}
models = {('MLP', MLPRegressor()),('RFR', RandomForestRegressor()),
          ('SVR', SVR()), ('AdaB', AdaBoostRegressor())}



In [ ]:
#!pip install matplotlib==3.2.2

Each of the models is trained, the results are saved and compared visually.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

# Cross-validation training
for scoring in error_metrics:
    results = [] # store metrics results
    msg = []  # print summary of result
    names = []  # store name of the models
    print('Evaluation metric: ', scoring)
    for name, model in models:
        print('Model ', name)
        cross_validation = KFold(n_splits=num_folds, shuffle=False)
        cv_results = cross_val_score(model, X_train, y_train, cv=cross_validation, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        resume = (name, cv_results.mean(), cv_results.std())
        msg.append(resume)
    print(msg)

    # Compare results between algorithms
    fig = plt.figure()
    fig.suptitle('Compare metric result for algorithms: %s' %scoring)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Candidate models')
    ax.set_ylabel('%s' %scoring)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    plt.show()

    results = []


## *7. Ajuste de los hiperparámetros*.

Pasos para realizar el ajuste de los hiperparámetros:
*Especificar el modelo a ajustar
* Especificar una métrica a optimizar
* Definir los rangos de los parámetros de búsqueda: *parámetros*
* Asignar un método de validación: *KFold*
* Buscar los hiperparámetros con los datos de validación: *X_val*

In [ ]:
modelo = RandomForestRegressor()
scoring='r2'
params = {
    # Number of trees in random forest
    'n_estimators': [100, 500, 800, 1000],  # default=100
     # Maximum number of levels in tree
    'max_depth': [2, None],  #deafult = None
     # Method of selecting samples for training each tree
}


# Search for the best combination of hyperparameters
cross_validation = KFold(n_splits=5, shuffle=False)
my_cv = cross_validation.split(X_val)
gsearch = GridSearchCV(estimator=modelo, param_grid=params, scoring=scoring, cv=my_cv)
gsearch.fit(X_val, y_val)

# Print best Result
print("Best result: %f using the following hyperparameters %s" % (gsearch.best_score_, gsearch.best_params_))
means = gsearch.cv_results_['mean_test_score']
stds = gsearch.cv_results_['std_test_score']
params = gsearch.cv_results_['params']

## *8. Evaluación final del modelo*.

Por último, se realizan predicciones de generación fotovoltaica.

Métricas de evaluación:
  * RMSE
  * R2

    
El modelo ``fit()`` se entrena con los hiperparámetros óptimos encontrados en el apartado anterior y a continuación se realizan las predicciones. 

In [ ]:
final_model = RandomForestRegressor(n_estimators=800) ## train again with the winner model from the Grid Search
final_model.fit(X_train,y_train)  # Model training 
y_predict = final_model.predict(X_test)  # prediction calculation


In [ ]:
y_predict

Es necesario invertir el escalado ``MinmaxScaler()`` para evaluar nuestros resultados en la dimensión original.

In [ ]:
import math 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error


# Invert the scaling and plot the results
y_test_unsc = np.reshape(y_test, (len(y_test), 1))
y_test_inv = scaler_y.inverse_transform(y_test_unsc)

y_predict_uns = np.reshape(y_predict, (len(y_predict), 1))
y_predict_inv = scaler_y.inverse_transform(y_predict_uns)

In [ ]:
# Error RMSE de test  
math.sqrt(mean_squared_error(y_test_inv, y_predict_inv))

## Resultados gráficos obtenidos. 

In [ ]:
# Plot y_predict vs y_test

x = range(len(y_predict_inv))
plt.figure(figsize=(20,5))
plt.xlabel('Time', size=15)
plt.ylabel('Energy produced (kWh)', size=15)
plt.plot(x, y_predict_inv, alpha=0.4, color='blue', label='PV predict')
plt.plot(x, y_test_inv, alpha=0.4, color='red',  label='PV real')
plt.title('Prediction vs Real')
plt.legend()
plt.show()

### ¡Necesitamos hacer Zoom!

Si es necesario, instale la biblioteca Plotly ``!pip install plotly``.

In [ ]:
import plotly.graph_objects as go  # Importamos la librería de plotly

init = list(range(len(y_predict_inv)))
y_predict_plot = pd.DataFrame(data=y_predict_inv, index=init, columns=['predict'])
y_test_plot = pd.DataFrame(data=y_test_inv, index=init, columns=['test'])


# We create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=init, y=y_predict_plot['predict'][init],
                    mode='lines',
                    name='PV prediction'))
fig.add_trace(go.Scatter(x=init, y=y_test_plot['test'][init],
                     mode='lines', name='PV real'))


# We edit figure
fig.update_layout(autosize=False,
                  width=1000,
                    height=500,
                    title='Prediccion vs Real',
                   xaxis_title='Periods',
                   yaxis_title='Energy (kWh)')


fig.show()

### Características/atributos más importantes 

¿Qué características tienen más peso en este ejemplo? 

In [ ]:
# We print the feature ranking
importances = gsearch.best_estimator_.feature_importances_
std = np.std([tree.feature_importances_ for tree in gsearch.best_estimator_.estimators_], axis=0)
indices = np.argsort(importances)[::-1]
feat = X.columns
feat_or=[]
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]])+feat[indices[f]])
    feat_or.append(feat[indices[f]])

# We plot the weight of the features that matter most for RandomForest()
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), feat_or)
plt.xlim([-1, X.shape[1]])
plt.show()